In [1]:
import re
from sympy import symbols, sympify, log

database_string = '''
ELEMENT /-   ELECTRON_GAS              0.0000E+00  0.0000E+00  0.0000E+00!
ELEMENT VA   VACUUM                    0.0000E+00  0.0000E+00  0.0000E+00!
ELEMENT AL   FCC_A1                    2.6982E+01  4.5773E+03  2.8322E+01!
ELEMENT CA   FCC_A1                    4.0078E+01  6.1965E+03  4.1589E+01!

SPECIES  AL2CA  AL2CA1  !

$ ------------------------------------------------------------------------------
TYPE_DEFINITION % SEQ *!
DEFINE_SYSTEM_DEFAULT ELEMENT 2 !
DEFAULT_COMMAND DEF_SYS_ELEMENT VA /- !

$ ------------------------------------------------------------------------------
$ FUNCTION AL
$ ------------------------------------------------------------------------------

FUNCTION GHSERAG    298.15  -7209.512+118.202013*T-23.8463314*T*LN(T)-.001790585*T**2-3.98587E-07*T**3-12011*T**(-1);          
1234.93  Y -15095.252+190.266404*T-33.472*T*LN(T)+1.412E+29*T**(-9); 
3000   N !

$ This is to be stripped
$ This is to be stripped

FUNCTION GHSERAGG    300.00  -23.8463314*T*LN(T)
-.001790585*T**2-3.98587E-07*T**3-12011*T**(-1);          1234.93  Y
-15095.252+190.266404*T-33.472*T*LN(T)+1.412E+29*T**(-9); 3000   N !

FUNCTION GHSERAGG    300  -23.8463314*T*LN(T)
-.001790585*T**2-3.98587E-07*T**3-12011*T**(-1);          1234.93  Y
-15095.252+190.266404*T-33.472*T*LN(T)+1.412E+29*T**(-9); 3000   N !

$ This is to be stripped
$ This is to be stripped
$ This is to be stripped

'''


def remove_comments(string):
    database_lines = string.split('\n')
    database_lines_no_comments = ''

    for line in database_lines:
        if not line.startswith('$' or '\n'):
            database_lines_no_comments = database_lines_no_comments + line

    return database_lines_no_comments

def split_lines(string):
    return string.split('!')

database_string = remove_comments(database_string)
database_lines = split_lines(database_string)

database_lines[8]

'FUNCTION GHSERAG    298.15  -7209.512+118.202013*T-23.8463314*T*LN(T)-.001790585*T**2-3.98587E-07*T**3-12011*T**(-1);          1234.93  Y -15095.252+190.266404*T-33.472*T*LN(T)+1.412E+29*T**(-9); 3000   N '

In [25]:
class thermo_func:
    def __init__(self, function_string):
        function_line = re.compile(r'FUNCTION\s+(\w+)+\s+(\d+.\d+)?+(.*)')
        match = function_line.match(function_string)
        if match:
            self.func_name = match.group(1)
            self.start_temp = match.group(2)
            self.func_steps = match.group(3)
            self.func_temps, self.func_expressions = self._split_piecewise(self.func_steps)

    
    def _split_piecewise(self, string):
        steps = re.split(r'[;, ,Y]+', string)

        func_steps_expresions = []
        func_steps_temps = []

        for line in range(0, len(steps), 2):
            if (steps[line-1] != ('' or 'N')) and (steps[line] != ('' or 'N')) and len(steps[line]) > 0:
                func_steps_expresions.append(steps[line - 1])
                func_steps_temps.append(steps[line])

        return func_steps_temps, func_steps_expresions
    

    def _select_temp_step(self, T):
        experssions = self.func_expressions
        temperatures = self.func_temps

        for i, temperature in enumerate(temperatures):
            if T <= float(temperature):
                return(experssions[i])
                
        print('Error,  temperature above max of {}'.format(self.func_temps[-1]))


    def evaluate_function(self, temperature):
        # Define the variable
        temp = symbols('T')

        # Example string
        expression = self._select_temp_step(temperature)
        print(expression)

        # Replace 'LN' with 'log' for natural logarithm
        expression = expression.replace('LN', 'log')

        # Convert the string to a SymPy expression
        expr = sympify(expression)

        # Substitute a value for T
        return float(expr.subs(temp, temperature).evalf())
    

In [36]:
thermo_func_1 = thermo_func(database_lines[9])

thermo_func_1.evaluate_function(1000)

-23.8463314*T*LN(T)-.001790585*T**2-3.98587E-07*T**3-12011*T**(-1)


-166925.8046127075

In [10]:
# Define the variable
temp = symbols('T')

# Example string
expression = thermo_func_1.func_expressions[1]

# Replace 'LN' with 'log' for natural logarithm
expression = expression.replace('LN', 'log')

# Convert the string to a SymPy expression
expr = sympify(expression)

# Substitute a value for T
value = 2500  # Example value for T
result = expr.subs(temp, value).evalf()

print(result)

-194145.375173722
